# Training a smaller GPT-2

Having created the GPT model in [gpt.ipynb](./gpt.ipynb), it's time to try training it.

In [ ]:
import import_ipynb
import gpt # type: ignore
from enum import Enum
from torch.utils.data import Dataset, DataLoader
import json
import math
import mlflow
import numpy as np
import os
import urllib.request
import tiktoken
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torch.cuda as cuda
import torch.backends
from pathlib import Path
from functools import partial
from typing import TypedDict

tokenizer = tiktoken.get_encoding("gpt2")

def get_device() -> torch.device:
    if cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps:0")
    else:
        return torch.device("cpu")


## The mini config

While we're just setting things up and experimenting, it's better to set a lower context size. If we kept it at 1024, training would take up a lot more memory and a lot more time, slowing down development.

`GPT_CONFIG_MINI` sets a smaller context size of 256, which is enough to see things start to work.

In [2]:
GPT_CONFIG_MINI: gpt.GPTConfigDict = {**gpt.GPT_CONFIG_124M, "context_length": 256}

## Convenience Functions and Example

We're adding a few functions to make it easier to interact with the model:
- `text_to_token_ids`: represents the very first stage in running the model.
- `token_ids_to_text`: represents the very last stage in running the model.

Also, another quick example of how they work.

Some things that are new:
- `simplified_model.eval();` this disables the training functionality of the model, letting us run inference much faster. It will skip dropout and discard gradients. The semicolon just discards the return value, which seems pointless in this example.
- `squeeze` and `unsqueeze`: These add or remove (respectively) an empty dimension to the outside of a tensor.
  - squeeze: `[[x]]` -> `[x]` (usually to remove the batch dimension when batch_size=1)
  - unsqueeze: `[x]` => `[[x]]` (to add a batch dimension where batch_size=1)

In [3]:
def text_to_token_ids(text: str, tokenizer: tiktoken.Encoding, device:torch.device=get_device()) -> torch.Tensor:
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor.to(device)

def token_ids_to_text(token_ids: torch.Tensor, tokenizer: tiktoken.Encoding) -> str:
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

# example:
def untrained_example(start_context:str="Every effort moves you"):
    torch.manual_seed(123)
    model = gpt.GPTModel(GPT_CONFIG_MINI)
    model.to(get_device())
    model.eval(); # disables dropout and gradients
    tokenizer = tiktoken.get_encoding("gpt2")

    token_ids = gpt.generate_text_simple(
        model=model,
        idx=text_to_token_ids(start_context, tokenizer).to(get_device()),
        max_new_tokens=10,
        context_size=GPT_CONFIG_MINI["context_length"],
    )

    print("Output text (untrained):\n", token_ids_to_text(token_ids, tokenizer))

if __name__ == "__main__":
    untrained_example()

Output text (untrained):
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


## Dataset

`Dataset` is a class that resembles an iterator, but specialized for passing to
a Dataloader. Like an iterator, it exposes `__len__` and `__getitem__` methods.

The specialization is that it returns input-target _pairs_ rather than single
items. This is what makes it specifically useful for training.

In this case, it also encapsulates the tokenization step and implements the
length/stride logic, but that's not always the case.

In [4]:
class GPTDatasetV1(Dataset):
    def __init__(self, text: str, tokenizer: tiktoken.Encoding, max_length: int, stride: int):
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            start = i
            end = start + max_length
            input_chunk = token_ids[start:end]
            target_chunk = token_ids[start+1:end+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self) -> int:
        return len(self.input_ids)
    
    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        return self.input_ids[idx], self.target_ids[idx]


## Minimal training example

Initially I had written a big wrapper class that enclosed a `SimplifiedGPT` instance and encapsulated a ton of training logic.

That approach had some advantages and disadvantages, but in the end I think it was a mistake. It resulted in highly complected code
with more and more configuration and more and more conditionals, although it did save some amount of passing arguments around.

Instead, I think it's preferable to mainly focus on functions that operate on the `GPTModel` class and/or its output logits.

To start, I'm going to focus on the bare minimum to train a model on [The Verdict](https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt). That means I need to be able to do the following:

1. download the text of The Verdict and save it locally.
2. split text into training and validation portions, then return a `training_loader` and `validation_loader`.
3. measure a model's loss given a batch (i.e., a set of input-output pairs).
4. implement the minimal training flow using the above and a crucial call to `loss.backward()`.

### Training config

Training involves a lot of parameters, and they need to be referenced by a few
different functions. At the same time, most of them can have sensible defaults.

Rather than adding the parameters one by one to every function that might need them,
I'm going to create a dataclass that we can pass around.

In [40]:
class TrainingConfig(TypedDict):
    train_percent: float  # If the training corpus is text, the percentage to use for training. The rest is used for validation.
    max_length: int       # the maximum length of a given training batch
    stride: int           # the distance between starting points of training batches
    epochs: int           # the number of times to train on one set of data
    lr: float             # the learning rate used by the optimizer
    weight_decay: float   # the weight decay used by the optimizer
    temperature: float    # the temperature for token generation
    topk: int             # the number of logits to select for top-k token generation

def new_training_config(
        train_percent:float=0.9,
        max_length:int=1024,
        stride:int|None=None,
        epochs:int=1,
        lr:float=5e-4,
        weight_decay:float=0.1,
        temperature:float=0.8,
        topk:int=50,
) -> TrainingConfig:
    if stride is None:
        stride = max_length // 2
    return TrainingConfig(
        train_percent=train_percent,
        max_length=max_length,
        stride=stride,
        epochs=epochs,
        lr=lr,
        weight_decay=weight_decay,
        temperature=temperature,
        topk=topk,
    )


### Training on "The Verdict"

Training on this short story is probably the last time we'll pass a text file in
directly for training, so some of these functions will be short-lived.

The most important function here is `train_simple_text`, which illustrates in
the most minimal possible way how the training flow works. Most of that boils
down to this one little stanza:

```python
    for epoch in range(cfg['epochs']):
        model.train()
        for input_batch, target_batch in training_loader:
            optimizer.zero_grad()
            loss = cross_entropy_loss_for_batch(model, input_batch=input_batch, target_batch=target_batch)
            loss.backward()
            optimizer.step()
```

To break that down in excruciating detail:
1. each epoch gets an entire run through the data loader, so we have a top-level loop here.
2. `model.train()` puts the model into training mode, where it preserves gradients for back propagation.
3. then we loop throuhg input-target pairs from the training loader.
4. `optimizer.zero_grad()` clears the optimizer state so that we don't contaminate this batch with the results from a previous batch.
5. we calculate the loss next. It's common to think of the loss as a scalar score, but in this case it encapsulates the entire gradient.
6. hence, we can call `loss.backward()` to backpropagate and update the model's weights.
7. finally, `optimizer.step()` updates the optimizer's parameters for one step.


In [ ]:
# Download the text if it's not yet available, then return it as a string
def the_verdict() -> str:
    """Returns the text of the short story \"The Verdict\". Uses the local filesystem for caching."""
    file_path = Path("the-verdict.txt")
    if not file_path.exists():
        url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode('utf-8')
            with open(file_path, "w") as f:
                f.write(text_data)
            return text_data
    with open(file_path, "r") as f:
        return f.read()

def text_training_loaders(text: str, cfg: TrainingConfig) -> tuple[DataLoader, DataLoader]:
    """Turn the given text into two Dataloaders: one for training and one for validation."""
    split_idx = int(len(text) * cfg['train_percent'])
    
    # Use partials with the Dataset and Dataloader classes to declutter and enforce consistency
    custom_dataset = partial(GPTDatasetV1, tokenizer=tokenizer, max_length=cfg['max_length'], stride=cfg['stride'])
    custom_dataloader = partial(DataLoader, batch_size=4, shuffle=True, drop_last=True, num_workers=0)

    # raw text portions
    train_portion = text[:split_idx]
    validation_portion = text[split_idx:]

    # tokenized datasets
    train_dataset = custom_dataset(train_portion)
    validation_dataset = custom_dataset(validation_portion)

    # completed dataloaders
    train_loader = custom_dataloader(train_dataset)
    validation_loader = custom_dataloader(validation_dataset)
    return (train_loader, validation_loader)

def cross_entropy_loss_for_batch(model: gpt.GPTModel, input_batch: torch.Tensor, target_batch: torch.Tensor) -> torch.Tensor:
    """Returns the model's loss for the given batch. The loss can be used to train the model."""
    device = model.device()
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    return nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten()) # TODO: explain why flatten

def calc_loss_loader(model: gpt.GPTModel, data_loader: DataLoader, num_batches=None) -> float:
    """Calculates the model's total loss over a number of batches for the given
    Dataloader. This helper is used for validation only."""
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = cross_entropy_loss_for_batch(model, input_batch, target_batch)
            total_loss += loss.item()
        else:
            break

    return total_loss / num_batches

def train_simple_text(model: gpt.GPTModel, text: str, cfg: TrainingConfig) -> float:
    optimizer = optim.AdamW(
        model.parameters(), lr=cfg['lr'], weight_decay=cfg['weight_decay']
    )
    training_loader, validation_loader = text_training_loaders(text, cfg)

    for epoch in range(cfg['epochs']):
        model.train()
        for input_batch, target_batch in training_loader:
            optimizer.zero_grad()
            loss = cross_entropy_loss_for_batch(model, input_batch=input_batch, target_batch=target_batch)
            loss.backward()
            optimizer.step()

    model.eval()
    if len(validation_loader) == 0:
        raise ValueError("Ooops, no validation data")
    with torch.no_grad():
        validation_loss = calc_loss_loader(model, validation_loader)
        return validation_loss

def train_verdict(model: gpt.GPTModel) -> float:
    torch.manual_seed(123)
    text = the_verdict()
    verdict_training_config = new_training_config(train_percent=0.85, max_length=256, epochs=10)
    return train_simple_text(model=model, text=text, cfg=verdict_training_config)

### Test out the training loop

Below you'll see the untrained output, the final training loss, and the trained output.

The trained output looks _much_ better, but there is a little bit of a trick
here. Training 10 epochs on such a small and homogenous dataset means that we're
_massively_ overfitting. The model is basically just memorizing phrases from the text
and spitting them out.

In [39]:
def trained_example(model: gpt.GPTModel, start_context):
    torch.manual_seed(123)
    model.eval()
    tokenizer = tiktoken.get_encoding("gpt2")

    token_ids = gpt.generate_text_simple(
        model=model,
        idx=text_to_token_ids(start_context, tokenizer),
        max_new_tokens=10,
        context_size=GPT_CONFIG_MINI["context_length"],
    )

    print("Output text (trained):\n", token_ids_to_text(token_ids, tokenizer))

if __name__ == "__main__":
    start_context = "He never"
    untrained_example(start_context=start_context)
    model = gpt.GPTModel(GPT_CONFIG_MINI)
    model.to(get_device())
    val_loss = train_verdict(model)
    print(f"\nValidation loss after training: {val_loss:.3f}\n")
    trained_example(model, start_context=start_context)


Output text (untrained):
 He neverRPteness champagneceptionuffم refres Rexrunnerchapter

Validation loss after training: 6.562

Output text (trained):
 He never had lingered to give a lump of sugar to


# Better text generation

So far, we've been using `gpt.generate_text_simple`, which chooses the next token by just finding the logit with the highest score.

That's good enough for illustration early in the process, but it honestly sucks. It results in super boring and repetitive output.

Let's implement a more typical Top-K multinomial algorithm:
1. Limit our choices to the top k logits, where k is a constant of our choosing. This prevents extremely unlikely tokens from _ever_ being generated.
2. Divide the logit probabilities by `temperature`. A low temperature emphasizes logits with higher probabilities, a higher temperature creates chaos by emphasizing lower probabilities.
3. Randomly choose from the remaining logits, weighted by their respective probabilities. Logits with higher probabilities will be chosen more often, but even unlikely ones will be chosen sometimes.

In [42]:
END_OF_TEXT = 50256

def choose_from_topk(logits: torch.Tensor, topk: int, temperature: float) -> torch.Tensor:
    top_logits, top_pos = torch.topk(logits, topk)
    filtered = torch.full_like(
        logits, -torch.inf
    )
    filtered.scatter_(dim=1, index=top_pos, src=top_logits) #huh?
    scaled = filtered / temperature
    probabilities = torch.softmax(scaled, dim=-1) # note: might have trouble with device
    if torch.any(torch.isnan(probabilities)) or torch.any(probabilities < 0):
        print("Bad probabilities:", probabilities)
        print("Logits:", logits)
        raise ValueError("NaNs or invalid values in probabilities")
    return torch.multinomial(probabilities, num_samples=1)

def generate_text_topk(model: gpt.GPTModel, token_ids: torch.Tensor, max_new_tokens: int, context_size: int, topk: int, temperature: float):
    for _ in range(max_new_tokens):
        idx_cond = token_ids[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        idx_next = choose_from_topk(logits, topk, temperature)
        if idx_next.item() == END_OF_TEXT:
            break
        token_ids = torch.cat((token_ids, idx_next), dim=1)
    return token_ids

def text_completion_topk(model, initial_context: str, max_new_tokens:int=10, context_size:int=256, topk:int=50, temperature:float=1.5):
    device = model.device()
    encoded = tokenizer.encode(initial_context)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0).to(device)
    model.eval()
    out = generate_text_topk(
        model,
        encoded_tensor,
        context_size=context_size,
        max_new_tokens=max_new_tokens,
        topk=topk,
        temperature=temperature,
    )
    decoded_text = tokenizer.decode(out.squeeze(0).tolist())
    return decoded_text

In [48]:
if __name__ == "__main__":
    start_context = "He never"
    model = gpt.GPTModel(GPT_CONFIG_MINI)
    model.to(get_device())
    train_verdict(model)
    print(text_completion_topk(model, start_context, temperature=0.4))

He never thought him, so often, basking under similar


# Saving and loading model state



In [ ]:
def save(model: gpt.GPTModel, name: str, overwrite:bool=False, optimizer: optim.Optimizer|None=None):
    if len(name) == 0:
        raise ValueError("name can't be empty")
    path = Path(f"{name}.pth")
    if path.exists() and not overwrite:
        raise FileExistsError(f"{path} already exists and overwrite is set to False")

    optimizer_state = optimizer.state_dict() if optimizer is not None else None
    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer_state,
    }, path)

def load(model: gpt.GPTModel, name: str, optimizer: optim.Optimizer|None=None):
    if len(name) == 0:
        raise ValueError("name can't be empty")
    path = Path(f"{name}.pth")
    if not path.exists():
        raise FileNotFoundError(f"{path} does not exist")
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer_state_dict = checkpoint["optimizer_state_dict"]
    if optimizer_state_dict is not None and optimizer is not None:
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    

In [52]:
start_context = "He never"
model = gpt.GPTModel(GPT_CONFIG_MINI)
model.to(get_device())
train_verdict(model) # No access to optimizer at this point
save(model, "test-verdict")

In [55]:
model = gpt.GPTModel(GPT_CONFIG_MINI)
load(model, name="test-verdict")
print(text_completion_topk(model, start_context, temperature=0.4))

He never thought him, so often, basking under similar


In [8]:
class DecodingStrategy(Enum):
    Greedy = 1
    Multinomial = 2
    TopK = 3

class GPTTrainingConfig(TypedDict):
    n_epochs: int            # the number of times to iterate through the entire dataloader
    initial_lr: float        # the starting (minimum) learning rate
    peak_lr: float           # the maximum learning rate
    warmup_steps: int        # the number of steps it should take to ramp up from initial_lr to peak_lr.
                             #   defaults to len(loader) / n_epochs if 0.
    gradient_clipping: bool  # whether to apply norm-based gradient clipping with a max_norm of 1.0
    eval_frequency: int      # the model will be evaluated and saved every this number of batches
    name: str                # the base filename used for saving results. if empty, model will not be saved.
    weight_decay: float      # helps to keep the model from overfitting by penalizing large weights
    train_ratio: float       # in text mode, the portion of the text that is used
                             #   for training. the rest is reserved for validation.

DEFAULT_TRAINING_CONFIG: GPTTrainingConfig = {
    "n_epochs": 1,
    "initial_lr": 0.0001,
    "peak_lr": 0.01,
    "warmup_steps": 0,
    "gradient_clipping": False,
    "eval_frequency": 5,
    "train_ratio": 0.9,
    "weight_decay": 0.1,
    "name": "",
}

END_OF_TEXT = 50256

class GPTModel:
    def __init__(self, cfg: GPTConfigDict, training_cfg: GPTTrainingConfig, force_cpu:bool=False, model=None):
        self.cfg = cfg
        self.training_cfg = training_cfg
        self.name = training_cfg['name']
        self.train_ratio = training_cfg['train_ratio']
        self.eval_frequency = training_cfg['eval_frequency']
        self.tokenizer = tiktoken.get_encoding("gpt2")
        self.force_cpu = force_cpu
        self.device = self.get_device()
        self.gradient_clipping = training_cfg['gradient_clipping']
        if model == None:
            self.model = SimplifiedGPT(self.cfg).to(self.device)
        else:
            self.model = model.to(self.device)
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(),
            weight_decay=training_cfg['weight_decay'],
        )
        self.tokens_seen, self.global_step = 0, -1

    def warmup_steps(self, loader):
        cfg_val = self.training_cfg['warmup_steps']
        if cfg_val == 0:
            return len(loader) / self.training_cfg['n_epochs']
        return cfg_val

    def learning_rate(self, step, loader):
        warmup_steps = self.warmup_steps(loader)
        if step < warmup_steps:
            return self.training_cfg['initial_lr'] + step * self.lr_increment(loader)
        
        total_training_steps = len(loader) * self.training_cfg['n_epochs']
        min_lr = 0.1 * self.training_cfg['initial_lr']
        peak_lr = self.training_cfg['peak_lr']
        # The following amounts to a cosine curve starting at the peak_lr and tapering off to the min_lr
        # slowly by the time we get to 100% progress.
        if total_training_steps <= warmup_steps:
            # this code is nonsense when epochs == 1,
            # TODO come up with a better calculation in that case.
            total_training_steps = warmup_steps + 1
        progress = ((step - warmup_steps) / 
                    (total_training_steps - warmup_steps))
        return min_lr + (peak_lr - min_lr) * 0.5 * (
            1 + math.cos(math.pi * progress)
        )

    def lr_increment(self, loader):
        return (self.training_cfg['peak_lr'] - self.training_cfg['initial_lr']) / self.warmup_steps(loader)

    def save(self, name: str):
        if name == "":
            return
        torch.save({
            "model_state_dict": self.model.state_dict(),
            "optimizer_state_dict": self.optimizer.state_dict()
        },
        f"{name}.pth")

    def load(self, name: str):
        checkpoint = torch.load(f"{name}.pth")
        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    def get_device(self) -> torch.device:
        if self.force_cpu:
            return torch.device("cpu")
        if torch.cuda.is_available():
            return torch.device("cuda:0")
        elif torch.backends.mps.is_available():
            return torch.device("mps")
        else:
            return torch.device("cpu")

    def create_dataloader(self, text: str, batch_size:int=4, max_length:int=256, stride:int=128, shuffle:bool=True, drop_last:bool=True, num_workers:int=0) -> DataLoader:
        dataset = GPTDatasetV1(text, self.tokenizer, max_length, stride)
        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            drop_last=drop_last,
            num_workers=num_workers
        )

    def loss_for_batch(self, input_batch: torch.Tensor, target_batch: torch.Tensor) -> torch.Tensor:
        assert input_batch.max().item() < self.cfg['vocab_size']
        assert target_batch.max().item() < self.cfg['vocab_size']
        input_batch, target_batch = input_batch.to(self.device), target_batch.to(self.device)
        logits = self.model(input_batch)
        return nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten()).to(self.device)

    def calc_loss_loader(self, data_loader, num_batches=None) -> float:
        total_loss = 0
        if len(data_loader) == 0:
            return float("nan")
        elif num_batches is None:
            num_batches = len(data_loader)
        else:
            num_batches = min(num_batches, len(data_loader))
        
        for i, (input_batch, target_batch) in enumerate(data_loader):
            if i < num_batches:
                loss = self.loss_for_batch(input_batch, target_batch)
                total_loss += loss.item()
            else:
                break
        return total_loss / num_batches
    
    def loaders(self, text: str, max_length:int=0, stride:int=0) -> tuple[DataLoader, DataLoader]:
        split_idx = int(self.train_ratio * len(text))
        train_data = text[:split_idx]
        validation_data = text[split_idx:]
        if stride == 0:
            stride = self.cfg["context_length"]
        if max_length == 0:
            max_length = self.cfg["context_length"]
        train_loader = self.create_dataloader(
            train_data,
            batch_size=2,
            max_length=max_length,
            stride=stride,
            drop_last=True,
            shuffle=True,
            num_workers=0,
        )
        validation_loader = self.create_dataloader(
            validation_data,
            batch_size=2,
            max_length=max_length,
            stride=stride,
            drop_last=True,
            shuffle=True,
            num_workers=0
        )
        return (train_loader, validation_loader)
    
    def evaluate(self, validation_loader, epoch:int=0, prompt:str=""):
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
            validation_loss = self.calc_loss_loader(validation_loader)
        summary = {
            "validation_loss": validation_loss,
            "device_type": self.device.type,
            "epoch": epoch
        }
        if len(prompt) > 0:
            example_output = self.prompt(prompt)
            summary["example_output"] = example_output[len(prompt):]
        return summary

    def choose(self, strategy: DecodingStrategy, logits: torch.Tensor, temperature:float, k:int=50):
        match strategy:
            case DecodingStrategy.Greedy:
                probabilities = torch.softmax(logits, dim=-1)
                result = torch.argmax(probabilities, dim=-1, keepdim=True)
                return result
            case DecodingStrategy.Multinomial:
                scaled = logits / temperature
                probabilities = torch.softmax(scaled, dim=-1)
                result = torch.multinomial(probabilities, num_samples=1)
                return result
            case DecodingStrategy.TopK:
                # print_topk_logits(logits, self.tokenizer, k)
                top_logits, top_pos = torch.topk(logits, k)
                filtered = torch.full_like(
                    logits, -torch.inf
                )
                filtered.scatter_(dim=1, index=top_pos, src=top_logits) #huh?
                scaled = filtered / temperature
                probabilities = torch.softmax(scaled, dim=-1).to(self.device)
                if torch.any(torch.isnan(probabilities)) or torch.any(probabilities < 0):
                    print("Bad probabilities:", probabilities)
                    print("Logits:", logits)
                    raise ValueError("NaNs or invalid values in probabilities")
                # assert probabilities.device == self.device
                return torch.multinomial(probabilities, num_samples=1)

    def generate_text_simple(self, token_ids: torch.Tensor, max_new_tokens, context_size, temperature:float):
        self.model.to(self.device)
        for _ in range(max_new_tokens):
            idx_cond = token_ids[:, -context_size:]
            with torch.no_grad():
                logits = self.model(idx_cond)
            logits = logits[:, -1, :]
            idx_next = self.choose(DecodingStrategy.TopK, logits, temperature=temperature)
            if idx_next.item() == END_OF_TEXT:
                break
            token_ids = torch.cat((token_ids, idx_next), dim=1)
        return token_ids

    def prompt(self, text: str, max_tokens:int=10, temperature:float=0.8) -> str:
        encoded = self.tokenizer.encode(text)
        encoded_tensor = torch.tensor(encoded).unsqueeze(0).to(self.device)
        self.model.to(self.device)
        self.model.eval()
        out = self.generate_text_simple(
            encoded_tensor,
            max_tokens,
            self.cfg["context_length"],
            temperature=temperature,
        )
        decoded_text = self.tokenizer.decode(out.squeeze(0).tolist())
        return decoded_text

    def generate_and_print_sample(self, prompt: str) -> None:
        print(self.prompt(prompt))

    def fix_summary(self, summary):
        """Makes summaries a little nicer to look at by reordering keys and rounding numbers."""
        summary['training_loss'] = round(summary['training_loss'], 4)
        summary['validation_loss'] = round(summary['validation_loss'], 4)
        return { k: summary[k] for k in ["training_loss", "validation_loss", "tokens_seen", "example_output", "learning_rate"] }

    def train_loader(self, training_loader: DataLoader, validation_loader: DataLoader, evaluation_prompt: str, epochs:int=1, run_name=""):
        self.model.to(self.device)
        self.tokens_seen = 0
        loss_summaries = []
        lr = self.training_cfg['initial_lr']

        with mlflow.start_run():
            if len(run_name) > 0:
                mlflow.set_tag("mlflow.runName", run_name)
            mlflow.set_tracking_uri("http://localhost:5000")
            mlflow.log_param("lr", lr)
            mlflow.log_param("epochs", epochs)
            mlflow.log_metric("tokens_seen", self.tokens_seen, step=self.global_step)

            for epoch in range(epochs):
                self.model.train()
                for input_batch, target_batch in training_loader:
                    self.optimizer.zero_grad()
                    self.global_step += 1
                    lr = self.learning_rate(self.global_step, training_loader)
                    for param_group in self.optimizer.param_groups:
                        param_group["lr"] = lr
                    loss = self.loss_for_batch(
                        input_batch, target_batch
                    )
                    loss_val = loss.item()
                    loss.backward()
    
                    # Gradient clipping: ensure the norms of the model parameters
                    # are not greater than 1 (only after the warmup period).
                    if self.gradient_clipping and self.global_step < self.warmup_steps(training_loader):
                        torch.nn.utils.clip_grad_norm_(
                            self.model.parameters(), max_norm=1.0,
                        )
                    
                    self.optimizer.step()
                    self.tokens_seen += input_batch.numel()
    
                    mlflow.log_metric("lr", lr, step=self.global_step)
                    mlflow.log_metric("loss", loss_val, step=self.global_step)
                    mlflow.log_metric("tokens_seen", self.tokens_seen, step=self.global_step)
                    mlflow.log_metric("epoch", epoch, step=self.global_step)
    
                    if self.eval_frequency > 0 and self.global_step % self.eval_frequency == 0:
                        summary = self.evaluate(validation_loader, epoch, evaluation_prompt)
                        summary["training_loss"] = loss_val
                        summary["tokens_seen"] = self.tokens_seen
                        summary["learning_rate"] = lr
                        summary = self.fix_summary(summary)
                        print(summary)
                        loss_summaries.append(summary)
                        mlflow.log_metric("validation_loss", summary["validation_loss"], step=self.global_step)
                        mlflow.log_text(summary['example_output'], artifact_file=f"example_outputs/example_{int(time.time())}.txt")
                        self.save(self.name)
                        # Defensive: try to prevent OOM
                        torch.cuda.empty_cache()
        print(f"Training complete. Steps: {self.global_step}, tokens: {self.tokens_seen}")

    def train_text(self, text: str, max_length:int=0, stride:int=0, prompt:str="Hello, I am "):
        training_loader, validation_loader = self.loaders(text, max_length, stride)
        epochs = self.training_cfg['n_epochs']
        return self.train_loader(training_loader, validation_loader, prompt, epochs)


def print_topk_logits(logits: torch.Tensor, tokenizer, k=10):
    """
    logits: torch.Tensor of shape [vocab_size] or [1, vocab_size]
    tokenizer: tiktoken tokenizer
    k: number of top tokens to print

    Used for debugging the `choose` method above.
    """
    if logits.dim() == 2:
        logits = logits.squeeze(0)  # shape [vocab_size]
    
    topk_vals, topk_indices = torch.topk(logits, k)
    
    print(f"{'Rank':<4} {'Token':<12} {'Index':<6} {'Logit':<10}")
    print("-" * 40)
    for i in range(k):
        token_str = tokenizer.decode([topk_indices[i].item()])
        print(f"{i+1:<4} {repr(token_str):<12} {topk_indices[i].item():<6} {topk_vals[i].item():<10.4f}")

NameError: name 'GPTConfigDict' is not defined

# Training on Project Gutenberg

The following few sections include the code necessary to:
1. Preprocess the [deepmind/pg19](https://huggingface.co/datasets/deepmind/pg19) dataset and wrap it in a Dataset class that we can use for training.
2. Sample some batches from the dataset to see what typical text looks like.
3. Train a GPTModel with a context size of 512 on this corpus.

If you want to do this, better get a big cup of coffee and about 60 gigs of
space ready. Downloading the data from HuggingFace takes a while, then you have
to expand it and then the LazyTokenDatasetPG19 class will create a cache containing
tokenized versions of every text.

After that, training takes about 6-8 hours to reach a plateau on my machine with an NVidia 3080.

In [ ]:
import re
import glob
from pathlib import Path
from datasets import load_dataset
import textwrap

class LazyTokenDatasetPG19(Dataset):
    """Preprocesses the dataset (assumed to be deepmind/pg19!) by creating a
    directory './tokens' containing pre-tokenized versions of all books in the
    dataset. This takes a while the first time you run it (maybe 20 minutes),
    but after that it's just a few seconds.
    
    The initialized object is suitable for passing to Dataloader."""
    GUTENBERG_END_RE = re.compile(r"(?i)end of (the )?project gutenberg.*", re.DOTALL)
    TOO_MANY_NEWLINES_RE = re.compile(r"\n{3,}")
    LEADING_NEWLINES_RE = re.compile(r"^\n+")
    
    def __init__(self, context_len:int=256):
        super().__init__()
        self.context_len = context_len
        self.preprocess()
        self.file_paths = glob.glob("tokens/*.pt")
        self.samples: list[tuple[int, int]] = []
        print("Loading data from tokens directory")
        for i, path in enumerate(self.file_paths):
            length = torch.load(path, map_location="cpu").shape[0]
            for j in range(0, length - context_len, context_len):
                self.samples.append((i, j))
            if i % 5_000 == 0:
                print(f"Loaded up to book {i}...")
        print("Loading complete")

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        book_idx, start = self.samples[idx]
        tokens = torch.load(self.file_paths[book_idx], map_location="cpu")
        input_ids = tokens[start : start + self.context_len]
        target_ids = tokens[start + 1 : start + self.context_len + 1]
        return input_ids, target_ids

    def preprocess(self):
        ds = load_dataset("deepmind/pg19", split="train")
        os.makedirs("tokens", exist_ok=True)
        existing_filepaths = glob.glob("tokens/book_*.pt")
        if len(existing_filepaths) >= (ds.num_rows - 5_000): # type: ignore[attr-defined]
            print("Preprocessing not needed.")
            return
        print("Preprocessing data to tokens directory.")
        tokenizer = tiktoken.get_encoding("gpt2")
        for i, book in enumerate(ds):
            path = Path(f"tokens/book_{i}.pt")
            if path.exists():
                continue
            text = self.clean_text(book['text'])
            if len(text) < self.context_len + 1:
                continue
            tokens = tokenizer.encode(text)
            torch.save(torch.tensor(tokens, dtype=torch.long), path)
            if i % 1_000 == 0:
                print(f"Completed preprocessing book {i}")

    def clean_text(self, text: str) -> str:
        # Remove Gutenberg end matter
        text = self.GUTENBERG_END_RE.split(text)[0]
        # Remove leading newlines/whitespace
        text = self.LEADING_NEWLINES_RE.sub("", text)
        # Collapse 3+ newlines into exactly 2 (paragraph break)
        text = self.TOO_MANY_NEWLINES_RE.sub("\n\n", text)
        # Eliminate chapter:verse markings
        text = re.sub(r'\b\d+:\d+\b', '', text)
        # Unwrap lines in each paragraph, but preserve paragraphs
        paragraphs = text.split('\n\n')
        unwrapped_paragraphs = [re.sub(r"\n", " ", p) for p in paragraphs]
        text = '\n\n'.join(unwrapped_paragraphs)
        # don't allow multiple spaces in a row
        text = re.sub(r"[^\S\n]+", " ", text)
        return text.strip()


In [ ]:
def sample_loader(dataloader, n):
    """Given a DataLoader and a number of samples, prints batches from the DataLoader."""
    tokenizer = tiktoken.get_encoding("gpt2")
    i = n
    for input_batch, target_batch in dataloader:
        if i == 0:
            break
        i -= 1
        text = tokenizer.decode(input_batch.tolist()[:64])
        print(text)
        print("----------------------------------------")

# Uncomment below to see what's in the sanitized pg19 dataset.
# sample_loader(DataLoader(ltds, shuffle=True)

In [ ]:
GPT_CONFIG_MEDIUM: GPTConfigDict = {**GPT_CONFIG_124M, "context_length": 512} # 1024 is just too big to train locally
ltds = None
training_cfg: GPTTrainingConfig = {**DEFAULT_TRAINING_CONFIG, "n_epochs": 20, "eval_frequency": 500, "name": "pg19_medium"}
model = GPTModel(GPT_CONFIG_MEDIUM, training_cfg)

def train_pg19(name: str, force_refresh:bool=False):
    walden = load_dataset("deepmind/pg19", split='validation')[0]['text'[63:]] # type: ignore
    if ltds == None or force_refresh: # type: ignore
        ltds = LazyTokenDatasetPG19(context_len=512)
    if force_refresh:
        model.load(name)
    training_loader = DataLoader(
        ltds, # type: ignore
        shuffle=True,
        batch_size=4,
    )
    model.train_loader(training_loader=training_loader, evaluation_text=walden, evaluation_prompt="When I", epochs=1) # type: ignore

# Uncomment below to actually train the model. You won't get good results until you do.
# model.load("pg19_medium")
# train_pg19("new_training_run")

In [ ]:
def prompt(model: GPTModel, txt:str, max_tokens=128, temperature=0.8):
    result = model.prompt(txt, max_tokens=max_tokens, temperature=temperature)
    print(textwrap.fill(result, width=120))

In [ ]:
# If the model is trained, you should see some... interesting results from this.
# Otherwise it'll just be gibberish.
# prompt(model, "Ere thrice the sun done salutation to the dawn,")